# EXECUTE IMMEDIATE - Dynamic ES|QL

This notebook demonstrates `EXECUTE IMMEDIATE` - the ability to build and execute ES|QL queries dynamically at runtime.

## Why Dynamic Queries?

Static queries are written at development time, but many use cases require:
- **Dynamic index names** - Query different indices based on date or tenant
- **Parameterized filters** - Apply user-provided filter values
- **Conditional aggregations** - Build different aggregations based on context
- **Schema flexibility** - Work with varying field names across indices

## Syntax

```sql
EXECUTE IMMEDIATE query_expression [INTO var1, var2, ...] [USING bind1, bind2, ...];
```

- `query_expression` - Any expression that evaluates to an ES|QL query string
- `INTO` - Optional: assign results to variables
- `USING` - Optional: bind variables (:1, :2, etc.) are replaced with provided values

## 1. Simple Dynamic Query

Execute a query stored in a variable:

In [ ]:
%%elastic_script

CREATE PROCEDURE simple_dynamic_query()
BEGIN
    DECLARE query STRING = 'FROM logs-sample | LIMIT 5';
    DECLARE results ARRAY;
    
    EXECUTE IMMEDIATE query INTO results;
    
    PRINT 'Found ' || ARRAY_LENGTH(results) || ' records';
END PROCEDURE

CALL simple_dynamic_query();

## 2. Dynamic Index Name

Build queries with dynamic index patterns:

In [ ]:
%%elastic_script

CREATE PROCEDURE query_dynamic_index(index_name STRING)
BEGIN
    DECLARE query STRING;
    DECLARE results ARRAY;
    
    SET query = 'FROM ' || index_name || ' | LIMIT 5';
    PRINT 'Executing: ' || query;
    
    EXECUTE IMMEDIATE query INTO results;
    PRINT 'Results: ' || ARRAY_LENGTH(results) || ' records';
END PROCEDURE

CALL query_dynamic_index('logs-sample');
CALL query_dynamic_index('metrics-sample');

## 3. Bind Variables with USING

Use positional bind variables (:1, :2, etc.) for safe parameter injection:

In [ ]:
%%elastic_script

CREATE PROCEDURE query_with_binds(level STRING, limit_count NUMBER)
BEGIN
    DECLARE results ARRAY;
    
    -- :1 will be replaced with level, :2 with limit_count
    EXECUTE IMMEDIATE 'FROM logs-sample | WHERE level = :1 | LIMIT :2'
        INTO results
        USING level, limit_count;
    
    PRINT 'Found ' || ARRAY_LENGTH(results) || ' ' || level || ' logs';
END PROCEDURE

CALL query_with_binds('ERROR', 10);
CALL query_with_binds('INFO', 5);

## 4. Dynamic Aggregations

Build aggregation queries dynamically:

In [ ]:
%%elastic_script

CREATE PROCEDURE count_by_field(index_name STRING, group_field STRING)
BEGIN
    DECLARE query STRING;
    DECLARE results ARRAY;
    
    SET query = 'FROM ' || index_name || ' | STATS count = COUNT(*) BY ' || group_field;
    PRINT 'Executing: ' || query;
    
    EXECUTE IMMEDIATE query INTO results;
    
    FOR row IN results LOOP
        PRINT row;
    END LOOP
END PROCEDURE

CALL count_by_field('logs-sample', 'level');

## 5. Multi-Variable INTO Clause

Extract individual values from query results:

In [ ]:
%%elastic_script

CREATE PROCEDURE get_stats(index_name STRING)
BEGIN
    DECLARE total_count NUMBER;
    DECLARE query STRING;
    
    SET query = 'FROM ' || index_name || ' | STATS total = COUNT(*)';
    EXECUTE IMMEDIATE query INTO total_count;
    
    PRINT index_name || ' has ' || total_count || ' documents';
END PROCEDURE

CALL get_stats('logs-sample');
CALL get_stats('users-sample');

## 6. Conditional Query Building

Build different queries based on conditions:

In [ ]:
%%elastic_script

CREATE PROCEDURE conditional_query(include_filter BOOLEAN, filter_value STRING)
BEGIN
    DECLARE query STRING;
    DECLARE results ARRAY;
    
    SET query = 'FROM logs-sample';
    
    IF include_filter THEN
        SET query = query || ' | WHERE level = :1';
    END IF
    
    SET query = query || ' | LIMIT 10';
    PRINT 'Query: ' || query;
    
    IF include_filter THEN
        EXECUTE IMMEDIATE query INTO results USING filter_value;
    ELSE
        EXECUTE IMMEDIATE query INTO results;
    END IF
    
    PRINT 'Found ' || ARRAY_LENGTH(results) || ' records';
END PROCEDURE

CALL conditional_query(true, 'ERROR');
CALL conditional_query(false, '');

## 7. Looping Over Dynamic Queries

Execute multiple dynamic queries in a loop:

In [ ]:
%%elastic_script

CREATE PROCEDURE multi_index_summary()
BEGIN
    DECLARE indices ARRAY = ['logs-sample', 'metrics-sample', 'users-sample'];
    DECLARE query STRING;
    DECLARE count NUMBER;
    
    FOR idx IN indices LOOP
        SET query = 'FROM ' || idx || ' | STATS count = COUNT(*)';
        EXECUTE IMMEDIATE query INTO count;
        PRINT idx || ': ' || count || ' documents';
    END LOOP
END PROCEDURE

CALL multi_index_summary();

## 8. Error Handling with Dynamic Queries

Handle errors when executing dynamic queries:

In [ ]:
%%elastic_script

CREATE PROCEDURE safe_dynamic_query(query STRING)
BEGIN
    DECLARE results ARRAY;
    
    TRY
        EXECUTE IMMEDIATE query INTO results;
        PRINT 'Success: ' || ARRAY_LENGTH(results) || ' results';
    CATCH
        PRINT 'Query failed: ' || error['message'];
    END TRY
END PROCEDURE

CALL safe_dynamic_query('FROM logs-sample | LIMIT 5');
CALL safe_dynamic_query('FROM nonexistent-index | LIMIT 5');

## Summary

`EXECUTE IMMEDIATE` provides:

1. **Dynamic query building** - Construct queries at runtime using string concatenation
2. **Bind variables** - Safe parameter injection with :1, :2, :3 syntax
3. **INTO clause** - Capture results into variables
4. **Full ES|QL support** - Any valid ES|QL query can be executed
5. **Error handling** - Works with TRY/CATCH for robust error management

### Best Practices

- Use **bind variables** for user-provided values (prevents injection)
- Validate index names and field names before building queries
- Use **TRY/CATCH** for production code
- Log queries during development for debugging
- Keep dynamic portions minimal - only parameterize what needs to vary